In [1]:
# import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.datasets

In [2]:
# simple cnn (2 convolution layers with a pool layers and a fully connected layer)
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (3,3), stride = (1,1), padding = (1,1))
        self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (3,3), stride = (1,1), padding = (1,1))
        self.fc = nn.Linear(16*7*7, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x

In [3]:
# set device
device = torch.device("cpu")

In [4]:
# hyperparameters
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 15

In [5]:
# data transformers
train_transformations = transforms.Compose([
    transforms.RandomRotation(degrees=(-45, 45)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

test_transformations = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

In [6]:
# functions to create loader
def train_load_dataset(data_path):
    dataset = datasets.ImageFolder(
        root=data_path,
        transform=train_transformations
    )
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=True
    )
    return loader

def test_load_dataset(data_path):
    dataset = datasets.ImageFolder(
        root=data_path,
        transform=test_transformations
    )
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=True
    )
    return loader

In [7]:
# load in the data
trainLoader = train_load_dataset('mnist_png/training')
testLoader = test_load_dataset('mnist_png/testing')

In [8]:
# check shape of torch tensor
examples = enumerate(train_load_dataset('mnist_png/training'))
batch_idx, (example_data, example_targets) = next(examples)
print(example_data.shape)

torch.Size([64, 1, 28, 28])


In [9]:
# instantiate the model
model = CNN().to(device)

In [10]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(trainLoader):
        
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent step
        optimizer.step()

In [12]:
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        
    model.train()

In [13]:
# training accuracy
check_accuracy(trainLoader, model)

Got 58352 / 60000 with accuracy 97.25


In [14]:
# testing accuracy
check_accuracy(testLoader, model)

Got 9784 / 10000 with accuracy 97.84
